# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
J. Klevas  ->  J. Klevas  |  ['J. Klevas']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
K. A. Kahle  ->  K. A. Kahle  |  ['K. A. Kahle']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 79 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.03653


extracting tarball to tmp_2510.03653...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.04677


extracting tarball to tmp_2510.04677...

...

Found 118 bibliographic references in tmp_2510.04953/HOWLS_KPpaper2.bbl.
Issues with the citations
syntax error in line 814: '}' expected
Retrieving document from  https://arxiv.org/e-print/2510.05101


extracting tarball to tmp_2510.05101...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


Found 54 bibliographic references in tmp_2510.05101/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.04677-b31b1b.svg)](https://arxiv.org/abs/2510.04677) | **Edge-On Disk Study (EODS) II: HCO$^+$ and CO vertical stratification in the disk surrounding SSTTau042021**  |
|| C. Foucher, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2025-10-07*|
|*Comments*| **|
|**Abstract**|            Context. Edge-on disks offer a unique opportunity to directly examine their vertical structure, providing valuable insights into planet formation processes. We investigate the dust properties, as well as the CO and HCO$^+$ gas properties, in the edge-on disk surrounding the T Tauri star 2MASS J04202144+281349 (SSTTau042021). Aims. We estimate the radial and vertical temperature and density profile for the gas and the dust. Methods. We use ALMA archival data of CO isotopologues and continuum emission at 2, 1.3 and 0.9 mm together with new NOEMA HCO$^+$ 3-2 observations. We retrieve the gas and dust disk properties using the tomographic method and the \textsc{DiskFit} model. Results. The vertical CO emission appears very extended, partly tracing the H$_2$ wind observed by JWST. C$^{18}$O, $^{13}$CO and HCO$^+$ emission characterize the bulk of the molecular layer. The dust and gas have a mid-plane temperatures of $\sim 7-11$ K. The temperature of the molecular layer (derived from $^{13}$CO and HCO$^+$) is on the order of 16 K. HCO$^+$ 3-2 being thermalized, we derive a lower limit for the H$_2$ volume density of $\sim 3 \times 10^6$ cm$^{-3}$ at radius 100-200 au between 1 and 2 scale heights. The atmosphere temperature of the CO gas is of the order $\sim$ 31 K at a radius of 100 au. We directly observe CO and HCO$^+$ gas onto the mid-plane beyond the dust outer radius ($\ge 300$ au). The (gas+dust) disk mass estimated up to a radius of 300 au is on the order of $4.6 \times 10^{-2} \mathrm{M}_\odot$. Conclusions. Thanks to the favorable disk inclination, we present the first quantitative evidence for vertical molecular stratification with direct observation of CO and HCO$^+$ gas along the mid-plane. We estimate the temperature profile with temperature of 7-11 K near the mid-plane, and 15-20 K in the dense part of the molecular layer up to $\sim$ 35 K above.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.04953-b31b1b.svg)](https://arxiv.org/abs/2510.04953) | **Euclid preparation: Towards a DR1 application of higher-order weak lensing statistics**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-10-07*|
|*Comments*| **|
|**Abstract**|            This is the second paper in the HOWLS (higher-order weak lensing statistics) series exploring the usage of non-Gaussian statistics for cosmology inference within \textit{Euclid}. With respect to our first paper, we develop a full tomographic analysis based on realistic photometric redshifts which allows us to derive Fisher forecasts in the ($\sigma_8$, $w_0$) plane for a \textit{Euclid}-like data release 1 (DR1) setup. We find that the 5 higher-order statistics (HOSs) that satisfy the Gaussian likelihood assumption of the Fisher formalism (1-point probability distribution function, $\ell$1-norm, peak counts, Minkowski functionals, and Betti numbers) each outperform the shear 2-point correlation functions by a factor $2.5$ on the $w_0$ forecasts, with only marginal improvement when used in combination with 2-point estimators, suggesting that every HOS is able to retrieve both the non-Gaussian and Gaussian information of the matter density field. The similar performance of the different estimators\inlinecomment{, with a slight preference for Minkowski functionals and 1-point probability distribution function,} is explained by a homogeneous use of multi-scale and tomographic information, optimized to lower computational costs. These results hold for the $3$ mass mapping techniques of the \textit{Euclid} pipeline: aperture mass, Kaiser--Squires, and Kaiser--Squires plus, and are unaffected by the application of realistic star masks. Finally, we explore the use of HOSs with the Bernardeau--Nishimichi--Taruya (BNT) nulling scheme approach, finding promising results towards applying physical scale cuts to HOSs.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05101-b31b1b.svg)](https://arxiv.org/abs/2510.05101) | **From theory to observation: understanding filamentary flows in high-mass star-forming clusters**  |
|| M. R. A. Wells, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-10-07*|
|*Comments*| *13 pages, 14 figures, accepted for publication in A&A*|
|**Abstract**|            Here we use data from multi-scale galactic MHD simulations to observe filaments and star forming clumps on 10's of pc scales and investigate flow rate relationships along, and onto filaments as well as flows towards the clumps. Using the FilFinderPPV identification technique, we identify the prominent filamentary structures in each data cube. Each filament and its corresponding clump are analysed by calculating flow rates along each filament towards the clump, onto each filament from increasing distances, and radially around each clump. This analysis is conducted for two cubes, one feedback dominated region, and one with less feedback. Looking at the face-on inclination of the simulations (0 degrees), we observe different trends depending on the environmental conditions (more or less feedback). The median flow rate in the region with more feedback is 8.9$\times$10$^{-5}$ M$_{sun}\mathrm{yr}^{-1}$ and we see that flow rates along the filaments toward the clumps generally decrease in these regions. In the region with less feedback we have a median flow rate of 2.9$\times$10$^{-4}$ M$_{sun}\mathrm{yr}^{-1}$ and when looking along the filaments here we see the values either increase or remain constant. We find that the flow rates from the environments onto the primary filaments are of an order of magnitude sufficient to sustain the flow rates along these filaments. When discussing the effects of galactic and filamentary inclination, we also observe that viewing the filaments from different galactic inclinations can reveal the presence of feeder structures (smaller filamentary structures aiding in the flow of material). The method used to estimate these flow rates, which has been previously applied to observational data, produced results consistent with those obtained from the simulations themselves, providing high confidence in the flow rate calculation method.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.03653-b31b1b.svg)](https://arxiv.org/abs/2510.03653) | **CCAT: Magnetic Sensitivity Measurements of Kinetic Inductance Detectors**  |
|| B. J. Vaughan, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-07*|
|*Comments*| **|
|**Abstract**|            The CCAT Observatory is a ground-based submillimeter to millimeter experiment located on Cerro Chajnantor in the Atacama Desert, at an altitude of 5,600 meters. CCAT features the 6-meter Fred Young Submillimeter Telescope (FYST), which will cover frequency bands from 210 GHz to 850 GHz using its first-generation science instrument, Prime-Cam. The detectors used in Prime-Cam are feedhorn-coupled, lumped-element superconducting microwave kinetic inductance detectors (KIDs). The telescope will perform wide-area surveys at speeds on the order of degrees per second. During telescope operation, the KIDs are exposed to changes in the magnetic field caused by the telescope's movement through Earth's magnetic field and internal sources within the telescope. We present and compare measurements of the magnetic sensitivity of three different CCAT KID designs at 100 mK. The measurements are conducted in a dilution refrigerator (DR) with a set of room temperature Helmholtz coils positioned around the DR. We discuss the implications of these results for CCAT field operations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.04722-b31b1b.svg)](https://arxiv.org/abs/2510.04722) | **Abundance of strontium in the Galactic globular cluster 47 Tuc**  |
|| E. Kolomiecas, A. Kučinskas, <mark>J. Klevas</mark>, V. Dobrovolskas |
|*Appeared on*| *2025-10-07*|
|*Comments*| *13 pages, 19 figures*|
|**Abstract**|            Aims. We have determined Sr abundance in a sample of 31 red giant branch stars located in the Galactic globular cluster 47 Tuc with the aim to identify potential differences in the Sr abundance between first population (1P, Na-poor) and second population (2P, Na-rich) stars. Methods. We derived the Na and Sr abundances from the archival spectra obtained with the UVES spectrograph. To do this, we used 1D ATLAS9 model atmospheres and a 1D local thermodynamic equilibrium spectral synthesis method. Particular attention was paid to assessing the potential impact of CN line blending on the obtained Sr abundances. Furthermore, we evaluated the potential influence of convection on the Sr line formation by using 3D hydrodynamical model atmospheres computed with the CO5BOLD code. Results. Our results suggest a weak correlation between the abundances of Sr and Na. Together with a similar correlation between the abundances of Zr and Na determined in our previous study, our analysis of Sr suggests that polluters that have enriched 2P stars with light elements may have produced some s-process elements as well. The mean Sr abundance determined in 31 red giant branch stars of 47~Tuc is $\langle {\rm [Sr/Fe]} \rangle = 0.18\pm0.08$ (the error denotes the standard deviation due to the star-to-star abundance scatter). This value is within the range of the Sr abundance variation that is observed in Galactic field stars of similar metallicity. The mean [Sr/Zr] abundance ratio in our sample stars suggests that the two s-process elements could have been synthesized by either low-mass asymptotic giant branch stars ($M=1-4 {\rm M}_{\odot}$) or massive ($M=10-20 {\rm M}_{\odot}$) fast-rotating ($v_{\rm rot}=200-300$ km/s) stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.04863-b31b1b.svg)](https://arxiv.org/abs/2510.04863) | **Heat Reveals What Clouds Conceal: Global Carbon & Longitudinally Asymmetric Chemistry on LTT 9779 b**  |
|| R. Ashtari, et al. -- incl., <mark>I. J. M. Crossfield</mark>, <mark>K. A. Kahle</mark>, <mark>L. Kreidberg</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-07*|
|*Comments*| *25 pages, 15 figures, 3 tables*|
|**Abstract**|            LTT-9779 b is an ultra-hot Neptune (Rp ~ 4.7 Re, Mp ~ 29 Me) orbiting its Sun-like host star in just 19 hours, placing it deep within the "hot Neptune desert," where Neptunian planets are seldom found. We present new JWST NIRSpec G395H phase-curve observations that probe its atmospheric composition in unprecedented detail. At near-infrared wavelengths, which penetrate the high-altitude clouds inferred from previous NIRISS/SOSS spectra, thermal emission reveals a carbon-rich atmosphere with opacity dominated by carbon monoxide (CO) and carbon dioxide (CO2). Both species are detected at all orbital phases, with retrieved mixing ratios of 10^-1 for CO and 10^-4 for CO2, indicating a globally well-mixed reservoir of carbon-bearing gases. We also moderately detect water vapor (H2O) and tentatively detect sulfur dioxide (SO2), providing insight into its chemistry and possible photochemical production under intense stellar irradiation. From these detections we infer a carbon-to-oxygen ratio near unity (C/O ~ 1) and a metallicity exceeding 500X Solar, consistent with equilibrium chemistry predictions for high-temperature atmospheres. This enrichment raises the mean molecular weight, reducing atmospheric escape, and likely helps LTT-9779 b retain a substantial atmosphere despite extreme irradiation. Our findings show that LTT-9779 b survives where few planets can, maintaining a carbon-rich atmosphere in a region where hot Neptune-class worlds are expected to evaporate. This makes LTT-9779 b a valuable laboratory for studying atmospheric escape and chemical processes under extreme conditions, offering new insight into the survival of planets in the hot Neptune desert.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.04677/./tomob_obs_12co21.png', 'tmp_2510.04677/./tomob_mod_12co21.png', 'tmp_2510.04677/./tomob_diff_12co21.png', 'tmp_2510.04677/./tomob_obs_12co32.png', 'tmp_2510.04677/./tomob_mod_12co32.png', 'tmp_2510.04677/./tomob_diff_12co32.png', 'tmp_2510.04677/./tomob_obs_13co.png', 'tmp_2510.04677/./tomo_mod_13co_def.png', 'tmp_2510.04677/./tomo_diff_13co_def.png', 'tmp_2510.04677/./tomob_obs_c18o.png', 'tmp_2510.04677/./tomob_mod_c18o.png', 'tmp_2510.04677/./tomob_diff_c18o.png', 'tmp_2510.04677/./tomob_obs_hco+.png', 'tmp_2510.04677/./tomob_mod_hco+.png', 'tmp_2510.04677/./tomob_diff_hco+.png', 'tmp_2510.04677/./hco_jwst.png', 'tmp_2510.04677/./13co21_jwst.png', 'tmp_2510.04677/./c18o_jwst.png', 'tmp_2510.04677/./12co21_jwst.png', 'tmp_2510.04677/./12cocont.png', 'tmp_2510.04677/./vel_chan_K_12co21.png', 'tmp_2510.04677/./vel_chan_K_12co32.png', 'tmp_2510.04677/./vel_chan_K_13co21.png', 'tmp_2510.04677/./vel_chan_K_c18o.png', 'tmp_2510.04677/./vel_chan_K_hco32.png'

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vlsr}{v_{\rm lsr}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\Msun}{ \mathrm{M}_\odot}$
$\newcommand{\Msunpyr}{\unit{M_\odot yr^{-1}}}$
$\newcommand{\coto}{\mbox{CO(2--1)}}$
$\newcommand{\cott}{\mbox{CO(3--2)}}$
$\newcommand{\tcott}{\mbox{^{13}CO(2--1)}}$
$\newcommand{\csff}{\mbox{CS(5--4)}}$
$\newcommand{\csss}{\mbox{CS(7--6)}}$
$\newcommand{\cnto}{\mbox{CN(2--1)}}$
$\newcommand{\cntt}{\mbox{CN(3--2)}}$
$\newcommand{\tablecontinuum}{$
$\begin{table}$
$\caption{Continuum Fit Parameters and Results}$
$\label{tab:disk}$
$\centering$
$\begin{tabular}{ p{4.5cm}|p{3cm}}$
$ \hline$
$Parameters  & Values \  \hline$
$ Inclination i_D & 86.6 \pm 0.1^\circ \  Inner radius  R_{Din}  & < 4 au \  Outer radius  R_{Dout}& 317 \pm 5 au\  T_{Do} = T_D({100 \mathrm{au}}) & 9.8 \pm 0.6 K \  Exponent of T_D & -0.3 \pm 0.2 \  \Sigma_{Do} = \Sigma_D({100 \mathrm{au}}) & 1.6\pm0.3 10^{23} cm^{-2} \  Exponent of \Sigma_D & 0.48 \pm 0.04 \ Dust emissivity index \beta  & 0.63 \pm 0.07  \ Absorption coefficient \kappa_0 & 0.052 cm^{2}/g \ \  \hline$
$ H_{Do} = H_D({100 \mathrm{au}}) & 11.6 au\  Exponent of H_D(r), h_d & -1.38  \   \hline$
$\end{tabular}$
$\tablefoot{\small{$
$Dust emissivity is \kappa(\nu) = \kappa_0 (\nu/345 \mathrm{GHz})^\beta. H_{Do} and h_D were fixed to the best fit values found from a simultaneous fit of the 220 and 330 GHz data only.}$
$}$
$  \label{tab:obtained-cont}$
$\end{table}$
$}$
$\newcommand{\tablemodel}{$
$\begin{table*}[!th]$
$\small$
$\caption{Best fit models for molecular lines}$
$\centering$
$\begin{tabular}{|c c|l|l|l|}$
$\hline$
$\multicolumn{2}{|c|}{Parameters} &  \multicolumn{1}{c|}{^{12}CO 2-1 \& 3-2} & \multicolumn{1}{c|}{^{13}CO 2-1 \& C^{18}O 2-1} &\multicolumn{1}{c|}{HCO^+ 3-2} \ \hline$
$\deltaV & (km s^{-1}) &  0.20 \pm 0.01  & 0.15 \pm 0.01 & 0.15 \ T_{atm}(r) & (K) & 31.2 \pm 0.2 (r/R_0)^{-0.68 \pm 0.02} & 16.8 \pm 0.2 (r/R_{0})^{0.07 \pm 0.01} & 15.0 \pm 0.2 (r/R_{0})^{(-0.12 \pm 0.03)} \ T_{mid}(r) & (K)&  \min(T_{atm},7.4 \pm 0.2  (r/R_{0})^{-0.27}) & \min(T_{atm},10.4 \pm 0.1  (r/R_{0})^{(-0.27 \pm 0.03)} & \min(T_{atm},9.1 \pm 0.4 (r/R_{0})^{-0.27} \ H(r)= & (au) & 27.9 \pm 0.2 (r/R_{0})^{1.16 \pm 0.02} & 24.0 \pm 0.2 (r/R_{0})^{1.25 \pm 0.03} & 30.8 \pm 0.8 (r/R_{0})^{0.93 \pm 0.13}\ \Sigma(r) & (cm^{-2}) & 8.2 \pm 1.2   10^{18} (r/R_{0})^{-3.1 \pm 0.2} & ^{13}CO : 4.5 \pm 0.4 10^{17} (r/R_{0})^{-3.3 \pm 0.2} & 6.1\pm 1.0  10^{13} (r/R_{0})^{-1.6 \pm 0.9} \  & &  & C^{18}O : 1.9 \pm 0.1 10^{16} (r/R_{0})^{-3.3 \pm 0.2} &\ \Sigma_{dep}   & (cm^{-2}) & 3.7\pm0.4 10^{22}  &  3.5\pm0.1  10^{22} & 8.7\pm0.2  10^{22} \ \hline$
$Inclination & (^\circ) & 85.5 \pm 0.6 & 89.8 \pm 0.4 & 87.4 \pm 0.8\ R_{in} & (au) &  61 \pm 1 & 77 \pm 2 & 53 \pm 1\ R_{out} & (au) & 760 \pm 5 & 790 \pm 6 & 521 \pm 2\  \hline$
$\end{tabular}$
$\label{tab:gas-model}$
$\tablefoot{Error bars are 1\sigma formal errors. All parameters are newcommandined in Appendix \ref{app:modeling}. Parameters with no error bars were fixed. The distance used is 160 pc and the V_\mathrm{LSR} is 7.35 km s^{-1}.$
$The mid-plane temperature exponent being loosely constrained, we fixed its value to that found from ^{13}CO 2-1 for all lines. Following Eq.\ref{Temperature}, we fix the exponent \delta that controls the steepness of the gradient to 2.5, see Appendix \ref{app:modeling}.$
$The pivot R_0 is fixed at 100 au to maximize the sensitivity. For all lines, we checked that the disk is in Keplerian rotation.}$
$\end{table*}$
$}$
$\newcommand{\figtomo}{$
$\begin{figure*}$
$  \subfigure(a){\hspace{5cm}}$
$  \subfigure(b){\hspace{5cm}}$
$  \subfigure(c) $
$  \centering$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_obs_12co21.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_mod_12co21.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_diff_12co21.pdf}}$
$  \centering$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_obs_12co32.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_mod_12co32.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_diff_12co32.pdf}}$
$  \centering$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_obs_13co.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomo_mod_13co_newcommand.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomo_diff_13co_newcommand.pdf}}$
$  \centering$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_obs_c18o.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_mod_c18o.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_diff_c18o.pdf}}$
$  \centering$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_obs_hco+.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_mod_hco+.pdf}}$
$  \subfigure{\includegraphics[width=0.268\textwidth]{tomob_diff_hco+.pdf}}$
$  \caption{From top to bottom: Tomographically Reconstructed Distribution (TRD) of ^{12}CO 2-1, ^{12}CO 3-2, ^{13}CO 2-1, C^{18}O 2-1 and HCO^+ 3-2 : (a) TRD of the observations (b) TRD of the final result of the models (c) difference between the observations and models TRD. For ^{12}CO (2–1), the residuals range from –2 to 2 K, while for the ^{12}CO (3–2) transition, they extend from –3 to 2 K. In the case of ^{13}CO (2–1), the differences are between 0 and 2 K. For C^{18}O (2–1), the deviations range from –1 to 2 K, and for HCO^+ (3–2), they span from –1 to 3 K. The vertical black line represents the effective resolution and two error bars show the widening of the beam as a function of radius caused by the TRD method.$
$  The contours are newcommandined from 4 K to 20 K in steps of 2 K for observational and model maps, and from -3 K to 3 K in steps of 1 K for the residual maps.}$
$\label{fig:tomo}$
$\end{figure*}$
$}$</div>



<div id="title">

# Edge-On Disk Study (EODS) II: HCO$^+$ and CO vertical stratification in the disk surrounding SSTTau042021

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.04677-b31b1b.svg)](https://arxiv.org/abs/2510.04677)<mark>Appeared on: 2025-10-07</mark> - 

</div>
<div id="authors">

C. Foucher, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Edge-on disks offer a unique opportunity to directly examine their vertical structure, providing valuable insights into planet formation processes. We investigate the dust properties, as well as the CO and HCO $^+$ gas properties, in the edge-on disk surrounding the T Tauri star 2MASS J04202144+281349 (SSTTau042021). We estimate the radial and vertical temperature and density profile for the gas and the dust. We use ALMA archival data of CO isotopologues and continuum emission at 2, 1.3 and 0.9 mm together with new NOEMA HCO $^+$ 3-2 observations. We retrieve the gas and dust disk properties using the tomographic method and the DiskFit model. The vertical CO emission appears very extended, partly tracing the $H_2$ wind observed by JWST. C $^{18}$ O, $^{13}$ CO and HCO $^+$ emission characterize the bulk of the molecular layer. The dust and gas have a mid-plane temperatures of $\sim 7-11$ K. The temperature of the molecular layer (derived from $^{13}$ CO and HCO $^+$ ) is on the order of 16 K. HCO $^+$ 3-2 being thermalized, we derive a lower limit for the $H_2$ volume density of $\sim 3 \times 10^6$ cm $^{-3}$ at radius 100-200 au between 1 and 2 scale heights. The atmosphere temperature of the CO gas is of the order $\sim$ 31 K at a radius of 100 au. We directly observe CO and HCO $^+$ gas onto the mid-plane beyond the dust outer radius ( $\ge 300$ au). The (gas+dust) disk mass estimated up to a radius of 300 au is on the order of $4.6\times10^{-2} \Msun$ . Thanks to the favorable disk inclination, we present the first quantitative evidence for vertical molecular stratification with direct observation of CO and HCO $^+$ gas along the mid-plane. We estimate the temperature profile with temperature of 7-11 K near the mid-plane, and 15-20 K in the dense part of the molecular layer up to $\sim$ 35 K above.

</div>

<div id="div_fig1">

<img src="tmp_2510.04677/./tomob_obs_12co21.png" alt="Fig2.1" width="6%"/><img src="tmp_2510.04677/./tomob_mod_12co21.png" alt="Fig2.2" width="6%"/><img src="tmp_2510.04677/./tomob_diff_12co21.png" alt="Fig2.3" width="6%"/><img src="tmp_2510.04677/./tomob_obs_12co32.png" alt="Fig2.4" width="6%"/><img src="tmp_2510.04677/./tomob_mod_12co32.png" alt="Fig2.5" width="6%"/><img src="tmp_2510.04677/./tomob_diff_12co32.png" alt="Fig2.6" width="6%"/><img src="tmp_2510.04677/./tomob_obs_13co.png" alt="Fig2.7" width="6%"/><img src="tmp_2510.04677/./tomo_mod_13co_def.png" alt="Fig2.8" width="6%"/><img src="tmp_2510.04677/./tomo_diff_13co_def.png" alt="Fig2.9" width="6%"/><img src="tmp_2510.04677/./tomob_obs_c18o.png" alt="Fig2.10" width="6%"/><img src="tmp_2510.04677/./tomob_mod_c18o.png" alt="Fig2.11" width="6%"/><img src="tmp_2510.04677/./tomob_diff_c18o.png" alt="Fig2.12" width="6%"/><img src="tmp_2510.04677/./tomob_obs_hco+.png" alt="Fig2.13" width="6%"/><img src="tmp_2510.04677/./tomob_mod_hco+.png" alt="Fig2.14" width="6%"/><img src="tmp_2510.04677/./tomob_diff_hco+.png" alt="Fig2.15" width="6%"/>

**Figure 2. -** From top to bottom: Tomographically Reconstructed Distribution (TRD) of $^{12}$CO 2-1, $^{12}$CO 3-2, $^{13}$CO 2-1, C$^{18}$O 2-1 and HCO$^+$ 3-2 : (a) TRD of the observations (b) TRD of the final result of the models (c) difference between the observations and models TRD. For $^{12}$CO (2–1), the residuals range from –2 to 2 K, while for the $^{12}$CO (3–2) transition, they extend from –3 to 2 K. In the case of $^{13}$CO (2–1), the differences are between 0 and 2 K. For C$^{18}$O (2–1), the deviations range from –1 to 2 K, and for HCO$^+$(3–2), they span from –1 to 3 K. The vertical black line represents the effective resolution and two error bars show the widening of the beam as a function of radius caused by the TRD method.
  The contours are defined from 4 K to 20 K in steps of 2 K for observational and model maps, and from -3 K to 3 K in steps of 1 K for the residual maps. (*fig:tomo*)

</div>
<div id="div_fig2">

<img src="tmp_2510.04677/./hco_jwst.png" alt="Fig5.1" width="20%"/><img src="tmp_2510.04677/./13co21_jwst.png" alt="Fig5.2" width="20%"/><img src="tmp_2510.04677/./c18o_jwst.png" alt="Fig5.3" width="20%"/><img src="tmp_2510.04677/./12co21_jwst.png" alt="Fig5.4" width="20%"/><img src="tmp_2510.04677/./12cocont.png" alt="Fig5.5" width="20%"/>

**Figure 5. -** Montage of JWST and interferometric data for SSTTau042021. MIRI MRS images from JWST reveal H$_{2}$ S(2) emission at 12.278 $\mu$m ([Arulanantham, McClure and Pontoppidan (2024)]()). White contours indicate the mid-infrared scattered-light continuum at 12.2 $\mu$m, and the white dashed line marks the rotation axis of the disk at PA = 73.5$^\circ$.
  From archival ALMA data and new NOEMA observations, cyan contours show molecular emission lines of HCO$^+$ 3–2, $^{13}$CO 2–1, C$^{18}$O 2–1, $^{12}$CO 2–1, and $^{12}$CO 3–2, while yellow contours represent the 0.85 mm continuum emission at 1 mJy/beam.  Beige circles in the bottom left corner of each panel represent the average theoretical FWHM of the PSF for JWST data, calculated from the relationship reported in [Law, Booth and Öberg (2023)](), and the beam of the molecular line emission is shown as a cyan ellipse in the top-right corner of each panel. The contours for HCO$^+$(3–2) range from 0.02 to 0.2 Jy/beam.km/s with a step of 0.02 Jy/beam.km/s, i.e 2.5 $\sigma$; for $^{13}$CO (2–1), they range from 0.02 to 0.08 Jy/beam.km/s with a step of 0.01 Jy/beam.km/s, i.e 3.5 $\sigma$; for C$^{18}$CO (2–1), they range from 0.008 to 0.016 Jy/beam.km/s with a step of 0.008 Jy/beam.km/s, i.e 3.5$ \sigma$; for $^{12}$CO (2–1), from 0.01 to 0.08 Jy/beam.km/s with a step of 0.02 Jy/beam.km/s, i.e 6 $\sigma$; and for $^{12}$CO (3–2) , from 0.06 to 0.8 Jy/beam.km/s with a step of 0.06 Jy/beam.km/s , i.e 4.2$\sigma$.
   (*fig:jwst*)

</div>
<div id="div_fig3">

<img src="tmp_2510.04677/./vel_chan_K_12co21.png" alt="Fig6.1" width="20%"/><img src="tmp_2510.04677/./vel_chan_K_12co32.png" alt="Fig6.2" width="20%"/><img src="tmp_2510.04677/./vel_chan_K_13co21.png" alt="Fig6.3" width="20%"/><img src="tmp_2510.04677/./vel_chan_K_c18o.png" alt="Fig6.4" width="20%"/><img src="tmp_2510.04677/./vel_chan_K_hco32.png" alt="Fig6.5" width="20%"/>

**Figure 6. -**  Velocity channel maps from top to bottom: for $^{12}$CO 2-1, 3-2, $^{13}$CO 2-1, C$^{18}$O 2-1 and HCO$^{+}$ 3-2. The channels corrupted by the CO clouds along the line of sight are marked with a red cross. These channels were not used in the DiskFit analysis. All the maps are rotated by $-73.5^\circ$. The contours are defined as follows: from 3 to 12 K in steps of 3 K (i.e., $3.6\sigma$) for the $^{12}$CO 2–1, $3.3\sigma$ for $^{12}$CO 3–2, and $3.6\sigma$ for $^{13}$CO 2–1; from 2 to 6 K in steps of 2 K (i.e., $3.2\sigma$) for C$^{18}$O 2–1; and from 3 to 9 K in steps of 3 K (i.e., $3.3\sigma$) for HCO$^+$ 3–2. (*fig:vel_channel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.04677"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\astroang}[1]{\ang[angle-symbol-over-decimal]{#1}}$
$\newcommand{\assign}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\inprogress}[1]{\textcolor[rgb]{1,0.5,0}{[#1]}}$
$\newcommand{\done}[1]$
$\newcommand{\R}[1]{\textcolor{red}{#1}}$
$\newcommand{\help}[1]{\textcolor{magenta}{[{\bf HELP!: #1}]}}$
$\newcommand{\nico}[1]{\textcolor{red}{[nico: #1]}}$
$\newcommand{\lucas}[1]{\textcolor{gray}{[lucas: #1]}}$
$\newcommand{\laila}[1]{\textcolor{magenta}{[laila: #1]}}$
$\newcommand{\JHD}[1]{\textcolor{magenta}{[JHD: #1]}}$
$\newcommand{\cora}[1]{\textcolor{purple}{[cora: #1]}}$
$\newcommand{\vincenzo}[1]{\textcolor{cyan}{[vincenzo: #1]}}$
$\newcommand{\lina}[1]{\textcolor{teal}{[Lina: #1]}}$
$\newcommand{\Sandrine}[1]{\textcolor{violet}{[Sandrine C.: #1]}}$
$\newcommand{\SP}[1]{\textcolor{teal}{[Sandrine P.: #1]}}$
$\newcommand{\ismael}[1]{\textcolor{brown}{[ismael: #1]}}$
$\newcommand{\sven}[1]{\textcolor{orange}{[sven: #1]}}$
$\newcommand{\simone}[1]{\textcolor{olive}{[Simone: #1]}}$
$\newcommand{\sihao}[1]{\textcolor{green}{[sihao: #1]}}$
$\newcommand{\filippo}[1]{\textcolor{cerulean}{[filippo: #1]}}$
$\newcommand{\cgiocoli}[1]{\textbf{\textcolor{brilliantlavender}{[cgiocoli: #1]}}}$
$\newcommand{\jeger}[1]{\textcolor{pink}{jeger: #1]}}$
$\newcommand{\alessandro}[1]{\textcolor{prune}{[alessandro: #1]}}$
$\newcommand{\natalia}[1]{\textcolor{cyan}{[natalia: #1]}}$
$\newcommand{\dani}[1]{\textcolor{teal}{[Daniela: #1]}}$
$\newcommand{\Francis}[1]{\textcolor{dgold}{[Francis: #1]}}$
$\newcommand{\ND}[1]{\textcolor{green}{[Nicolas D.: #1]}}$
$\newcommand{\andreas}[1]{\textcolor{kindofgreen}{[andreas: #1]}}$
$\newcommand{\MapMapMap}{\expval{\Map^3}}$
$\newcommand{\Mapn}{\expval{\Map^n}}$
$\newcommand{\varthetavec}{\vb*{\vartheta}}$
$\newcommand{\thetavec}{\vb*{\theta}}$
$\newcommand{\MapMap}{\expval{\Map^2}}$
$\newcommand{\ellvec}{\vb*{\ell}}$
$\newcommand{\inlinecomment}[1]$
$\newcommand{\orcid}[1]$
$\newcommand{\simone}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\filippo}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\vincenzo}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\Map}{M_\mathrm{ap}}$
$\newcommand{\arraystretch}{1.2}{$
$    \begin{tabular}{c c c c c c c}$
$        \hline$
$        \multicolumn{1}{c}{Statistics} & Abbreviation & Algorithm \         \hline$
$        shear 2-point correlation functions & \gamma-2PCFs (\xi_{+}/\xi_{-}) & \texttt{TreeCorr} \         convergence 2-point correlation function & \kappa-2PCF (\xi_{\kappa}) & \texttt{TreeCorr} \         1-point probability distribution function & PDF (\mathcal{P}) & \citetalias{Ajani-EP29},  \citet{Castiblanco:2024xnd} \         \rm \ell1 \! - \! norm & \rm \ell1 \! - \! norm & \citet{sreekanth2024theoreticalwaveletell1normonepoint} \         peak counts & peaks (N) & \texttt{lenspack} \         Minkowski functionals & MFs (V_0, V_1, V_2) & \citet{Vicinanza_2019} \         Betti numbers & BNs (\beta_0, \beta_1) & \citet{Parroni_2021} \         \hline$
$    \end{tabular}$
$    }$
$\newcommand{\tabcolsep}{2.5mm}{\newcommand{\arraystretch}{2.}$
$\begin{table}[ht]$
$\begin{minipage}{8.5cm}$
$\huge$
$\centering$
$\caption{\filippo{Rebinning and tail cuts of the non-tomographic fiducial DVs: DR1 optimistic is used as the fiducial n(z); KS is used as the fiducial mass reconstruction technique; 2.34 arcminuts is used as the fiducial top-hat smoothing scale (true ?). The second, third, fourth and fifth columns respectively correspond to the number of DV elements; the range of scales \theta, convergence \kappa, or SN \nu that is used for each DV; the observed SMAPE value; and the limit SMAPE value.}}\label{tab:tail_cuts}$
$\vspace{5pt}$
$\resizebox*{0.85\textwidth}{!}{$
$\begin{tabular}{c|c|c|c|c}$
$\hline$
$Statistics & N_{\rm d} & Range & \mathcal{S}_{\text{obs}} & \mathcal{S}_{\text{lim}} \ \hline$
$\xi_+ & 6 & 1'.57 < \theta < 29'.9  & \multirow{2}{*}{0.017} & \multirow{2}{*}{0.021} \ \xi_-$
$& 5 & 1'.57 < \theta < 16'.6 & & \ \hline$
$\kappa-2PCF & 4 & 2'.84 < \theta < 16'.6 & 0.018 & 0.021 \ \hline$
$PDF & 11 & -0.048 < \kappa < 0.033 & 0.018 & 0.021 \ \hline$
$\ell1-norm & \dots & \dots & \dots & \dots \ \hline$
$Peaks & \dots & \dots & \dots & \dots \ \hline$
$V_0 & \dots & \dots & \multirow{3}{*}{\dots} & \multirow{3}{*}{\dots} \ V_1 & \dots & \dots &  & \ V_2 & \dots & \dots & & \ \hline$
$\beta_0 & \dots & \dots & \multirow{2}{*}{\dots} & \multirow{2}{*}{\dots} \ \beta_1 & \dots & \dots &  & \ \hline$
$persistent BNs & \dots & \dots & \dots & \dots \ \hline$
$persistent heat. & \dots & \dots & \dots & \dots \ \hline$
$ST coef. & \dots & \dots & \dots & \dots \ \hline$
$\end{tabular}}$
$\end{minipage}$
$\end{table}}$
$\newcommand{\arraystretch}{2.}$</div>



<div id="title">

# $\Euclid$ preparation: Towards a DR1 application of higher-order weak lensing statistics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.04953-b31b1b.svg)](https://arxiv.org/abs/2510.04953)<mark>Appeared on: 2025-10-07</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** This is the second paper in the HOWLS (higher-order weak lensing statistics) series exploring the usage of non-Gaussian statistics for cosmology inference within _Euclid_ . With respect to our first paper, we develop a full tomographic analysis based on realistic photometric redshifts which allows us to derive Fisher forecasts in the ( $\sigma_8$ , $w_0$ ) plane for a _Euclid_ -like data release 1 (DR1) setup. We find that the 5 higher-order statistics (HOSs) that satisfy the Gaussian likelihood assumption of the Fisher formalism (1-point probability distribution function, $\ell$ 1-norm, peak counts, Minkowski functionals, and Betti numbers) each outperform the shear 2-point correlation functions by a factor $2.5$ on the $w_0$ forecasts, with only marginal improvement when used in combination with 2-point estimators, suggesting that every HOS is able to retrieve both the non-Gaussian and Gaussian information of the matter density field. The similar performance of the different estimators $\inlinecomment{, with a slight preference for Minkowski functionals and 1-point probability distribution function,}$ is explained by a homogeneous use of multi-scale and tomographic information, optimized to lower computational costs. These results hold for the $3$ mass mapping techniques of the _Euclid_ pipeline: aperture mass, Kaiser--Squires, and Kaiser--Squires plus, and are unaffected by the application of realistic star masks. Finally, we explore the use of HOSs with the Bernardeau--Nishimichi--Taruya (BNT) nulling scheme approach, finding promising results towards applying physical scale cuts to HOSs.

</div>

<div id="div_fig1">

<img src="tmp_2510.04953/./Plots/noBNT_vs_BNT_vs_BNT-smoothing.png" alt="Fig9" width="100%"/>

**Figure 9. -** Impact of BNT: from left to right, DR3-like unmasked Fisher contours inferred from $\gamma$-2PCFs, $\kappa$-2PCF and HOSs (smoothing scale $\ang{;2.34;}$) measured on KS-reconstructed mass maps of all tomographic slices excluding the non-tomographic configuration in no-BNT (orange), BNT (green) and BNT-smoothing (purple) setups. (*fig:no-BNT vs. BNT vs. BNT-smoothing*)

</div>
<div id="div_fig2">

<img src="tmp_2510.04953/./Plots/shear_2pcfs_kappa_2pcf_kappa_1pdf_kappa_peak_counts_kappa_bns_kappa_mfs_kappa_l1norm_s8_w_Nf_7616_Nc_256_contour_plot.png" alt="Fig12" width="100%"/>

**Figure 12. -** Fisher forecasts of reference setup with masks, full-tomography and multiscale, for $\gamma$-2PCFs (cyan) and HOSs (orange), as well as for their combination (green). For a qualitative inspection, contours of all statistics are overplotted in the last panel for varying colors. $\inlinecomment${$\cora${change title of 2pcf plot to remove $\kappa$ as it also shows $\gamma$.}$\cora${Can we remove the l1 norm here (saying it looks like the pdf as seen in previous figs) to gain more space and make the comparison plot less busy. also please remove contour shading at least in the all probes case.} $\simone${Addressed all comments: we keep l1-norm for completeness.}} (*fig:money_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2510.04953/./Plots/DR1_opt_single_probe_equi_pop_auto_4_x_4_scale_px_shear_2pcfs_s8_w_Nf_256_Nc_256_theory_DVs_plot.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\gamma$-2PCFs (upper panel) and derivatives (lower panels) DV-comparison for bin 5 between the mean computed from DUSTGRAIN mocks (orange) and theory predictions (black) for the fiducial cosmology (top), $\sigma_8$(middle) and $w_0$(bottom) derivatives. Fractional residuals are reported in purple on the secondary $y$-axis, emphasizing a 15\% threshold as a grey-shaded horizontal band. $\inlinecomment${$\jeger${The labels on the right hand side are cut off} $\simone${I did it on purpose to save space, it should be clear through the color-coding.}}$\inlinecomment${$\JHD${Only the top panel shows $\gamma$-2PCFs, the other panels show derivatives. Maybe have 'upper:' and 'lower:'? } $\simone${Agreed, added.}} (*fig:shear_2pcfs_theory_validation:DV_comparison_bin4*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.04953"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\flowrate}{~M_{\sun}\mathrm{yr}^{-1}}$</div>



<div id="title">

# From theory to observation: understanding filamentary flows in high-mass star-forming clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.05101-b31b1b.svg)](https://arxiv.org/abs/2510.05101)<mark>Appeared on: 2025-10-07</mark> -  _13 pages, 14 figures, accepted for publication in A&A_

</div>
<div id="authors">

M. R. A. Wells, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Filamentary structures on parsec scales play a critical role in feeding star-forming regions, often acting as the main channels through which gas flows into dense clumps that foster star formation. Understanding the dynamics of these filaments is crucial for explaining the mechanisms of star formation across a range of environments. Here we use data from multi-scale galactic MHD simulations to observe filaments and star forming clumps on 10's of pc scales and investigate flow rate relationships along, and onto filaments as well as flows towards the clumps. Using the \texttt{FilFinderPPV} identification technique, we identify the prominent filamentary structures in each data cube. Each filament and its corresponding clump are analysed by calculating flow rates along each filament towards the clump, onto each filament from increasing distances, and radially around each clump. This analysis is conducted for two cubes, one feedback dominated region, and one with less feedback, as well as for five different inclinations (0, 20, 45, 70, and 90 degrees) of one filament and clump system. Looking at the face-on inclination of the simulations (0 degrees), we observe different trends depending on the environmental conditions (more or less feedback). The median flow rate in the region with more feedback is 8.9 $\times$ 10 $^{-5}$ $\flowrate$ and we see that flow rates along the filaments toward the clumps generally decrease in these regions. In the region with less feedback we have a median flow rate of 2.9 $\times$ 10 $^{-4}$ $\flowrate$ and when looking along the filaments here we see the values either increase or remain constant. We find that the flow rates from the environments onto the primary filaments are of an order of magnitude sufficient to sustain the flow rates along these filaments. When discussing the effects of galactic and filamentary inclination, we also observe that viewing the filaments from different galactic inclinations can reveal the presence of feeder structures (smaller filamentary structures aiding in the flow of material). Additionally, considering the inclination of the filaments themselves allows us to determine how much we are overestimating or underestimating the flow rates for those filaments. The different trends in the relationship between flow rate and distance along the filaments in both the feedback and non-feedback dominated cubes confirm that the environment is a significant factor in accretion flows and their relationship with filament parameters. The method used to estimate these flow rates, which has been previously applied to observational data, produced results consistent with those obtained from the simulations themselves, providing high confidence in the flow rate calculation method.

</div>

<div id="div_fig1">

<img src="tmp_2510.05101/./images/along_q.png" alt="Fig11" width="100%"/>

**Figure 11. -** Flow rate vs. distance relation for the quiet cubeDistance vs flow rate, in log space, relationship for each filament in the quiet cube (filaments are labelled in Fig. \ref{fig:quietcubeskel}), colour coded by velocity difference. (*fig:quiet_along*)

</div>
<div id="div_fig2">

<img src="tmp_2510.05101/./images/along.png" alt="Fig10" width="100%"/>

**Figure 10. -** Flow rate vs. distance relation for the active cubeDistance vs flow rate, in log space, relationship for each filament in the active cube (filaments are labelled in Fig. \ref{fig:activecubeskel}), colour coded by velocity difference. (*fig:active_along*)

</div>
<div id="div_fig3">

<img src="tmp_2510.05101/./images/points.png" alt="Fig1" width="100%"/>

**Figure 1. -** Along, onto and polar points demonstration0th moment map of the Active column density cube with the identified filamentary structure, colour coded by velocity, overlaid on top. Green, red and blue points indicating the different types of flow rate, green is along the filamentary structure, red is onto, and blue is polar around the clumps. 2.85 pc scale bar in the top right corner.  (*fig:starpoints*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.05101"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

20  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
